In [57]:
import pandas as pd 
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import datetime

In [96]:
df_client = pd.read_csv('./data/predict-energy-behavior-of-prosumers/client.csv')
df_train = pd.read_csv('./data/predict-energy-behavior-of-prosumers/train.csv').sample(500000, random_state=4242)
df_gas_prices = pd.read_csv('./data/predict-energy-behavior-of-prosumers/gas_prices.csv')
df_electricity = pd.read_csv('./data/predict-energy-behavior-of-prosumers/electricity_prices.csv')
df_forecast_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/forecast_weather.csv')
df_historical_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/historical_weather.csv').sample(500000, random_state=4242)
df_weather_station = pd.read_csv('./data/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

In [97]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [98]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [99]:
df_client["date_str"] = df_client["date"].apply(lambda x : str(x))
df_electricity["date_str"] = df_electricity["date"].apply(lambda x : str(x))

Feature Engineering

In [100]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


Creation Colonne 

In [101]:
# crétaion de la colonne "temperature" avec une valeur initiale de -1
df_train["price_elec"] = -1

liste_date = df_train["datetime"].unique()

for i in range(len(liste_date)):
    if(i % 50 ==0):
        print(i, "/", len(liste_date), end="\r")
unique_date =liste_date[i]
for unique_date in df_train["datetime"].unique():
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]

C:\Users\Jawad\AppData\Local\Temp\ipykernel_14556\2816624801.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '199.99' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]


In [102]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,price_elec
1414989,11,0,3,790.836,1,2022-11-24 01:00:00,449,1414989,45,199.99
953684,9,1,3,4.396,0,2022-07-03 21:00:00,305,953684,37,211.31
103626,4,0,3,0.000,0,2021-10-06 05:00:00,35,103626,16,98.25
1745132,15,1,1,0.000,0,2023-03-06 19:00:00,551,1745132,59,129.23
1961232,7,1,1,0.000,0,2023-05-14 02:00:00,620,1961232,29,85.59
...,...,...,...,...,...,...,...,...,...,...
786780,0,0,2,0.000,0,2022-05-12 23:00:00,253,786780,1,54.10
914348,0,0,2,59.062,0,2022-06-21 15:00:00,293,914348,1,199.04
833853,4,1,0,902.756,1,2022-05-27 14:00:00,268,833853,17,109.55
188631,14,1,3,1935.450,1,2021-11-03 07:00:00,63,188631,56,119.38


Ajout de température 

In [103]:
df_historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id
43867,2021-09-17 07:00:00,7.3,4.3,0.0,0.00,1014.8,10,11,0,0,4.277778,79,28.0,13.0,15.0,59.1,24.2,17.0
1353413,2023-01-17 12:00:00,8.1,4.8,0.0,0.00,1005.8,85,94,0,0,9.916667,261,0.0,0.0,0.0,57.6,23.2,505.0
256936,2021-12-05 14:00:00,-12.3,-15.6,0.0,0.00,1007.3,88,98,0,0,4.055556,40,100.0,35.0,65.0,57.6,25.7,97.0
1358726,2023-01-19 11:00:00,0.8,-0.2,0.0,0.35,990.8,100,90,99,87,9.083333,259,28.0,0.0,28.0,58.5,26.7,507.0
1223014,2022-11-29 23:00:00,3.2,-0.8,0.1,0.00,1033.6,93,87,25,0,7.472222,305,0.0,0.0,0.0,59.4,21.7,456.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729574,2022-05-30 10:00:00,12.9,8.9,0.0,0.00,1007.1,100,87,55,84,4.027778,314,391.0,146.0,245.0,57.6,24.7,272.0
1179307,2022-11-13 17:00:00,6.0,4.6,0.0,0.00,1031.0,0,0,0,0,2.194444,133,0.0,0.0,0.0,58.8,22.2,440.0
1276505,2022-12-19 21:00:00,-6.8,-10.2,0.0,0.00,1019.4,60,27,41,36,4.250000,207,0.0,0.0,0.0,58.2,27.2,476.0
361291,2022-01-13 09:00:00,2.7,0.9,0.0,0.00,998.1,0,0,0,0,7.444444,187,0.0,0.0,0.0,59.4,25.2,135.0


In [104]:
df_historical_weather["datetime"].nunique()

df_historical_weather["date"] = pd.to_datetime(df_historical_weather["datetime"])
df_historical_weather["date"] = df_historical_weather["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_historical_weather["date"]

43867      17-09-21
1353413    17-01-23
256936     05-12-21
1358726    19-01-23
1223014    29-11-22
             ...   
729574     30-05-22
1179307    13-11-22
1276505    19-12-22
361291     13-01-22
1651931    08-05-23
Name: date, Length: 500000, dtype: object

In [105]:
df_historical_weather["date_str"] = df_historical_weather["date"].apply(lambda x : str(x))

In [106]:
df_historical_weather

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,date,date_str
43867,2021-09-17 07:00:00,7.3,4.3,0.0,0.00,1014.8,10,11,0,0,4.277778,79,28.0,13.0,15.0,59.1,24.2,17.0,17-09-21,17-09-21
1353413,2023-01-17 12:00:00,8.1,4.8,0.0,0.00,1005.8,85,94,0,0,9.916667,261,0.0,0.0,0.0,57.6,23.2,505.0,17-01-23,17-01-23
256936,2021-12-05 14:00:00,-12.3,-15.6,0.0,0.00,1007.3,88,98,0,0,4.055556,40,100.0,35.0,65.0,57.6,25.7,97.0,05-12-21,05-12-21
1358726,2023-01-19 11:00:00,0.8,-0.2,0.0,0.35,990.8,100,90,99,87,9.083333,259,28.0,0.0,28.0,58.5,26.7,507.0,19-01-23,19-01-23
1223014,2022-11-29 23:00:00,3.2,-0.8,0.1,0.00,1033.6,93,87,25,0,7.472222,305,0.0,0.0,0.0,59.4,21.7,456.0,29-11-22,29-11-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729574,2022-05-30 10:00:00,12.9,8.9,0.0,0.00,1007.1,100,87,55,84,4.027778,314,391.0,146.0,245.0,57.6,24.7,272.0,30-05-22,30-05-22
1179307,2022-11-13 17:00:00,6.0,4.6,0.0,0.00,1031.0,0,0,0,0,2.194444,133,0.0,0.0,0.0,58.8,22.2,440.0,13-11-22,13-11-22
1276505,2022-12-19 21:00:00,-6.8,-10.2,0.0,0.00,1019.4,60,27,41,36,4.250000,207,0.0,0.0,0.0,58.2,27.2,476.0,19-12-22,19-12-22
361291,2022-01-13 09:00:00,2.7,0.9,0.0,0.00,998.1,0,0,0,0,7.444444,187,0.0,0.0,0.0,59.4,25.2,135.0,13-01-22,13-01-22


In [107]:
# Ajouter la colonne "temperature" avec une valeur initiale de -1
df_train["temperature"] = -1

# Parcourir les dates uniques dans df_train
for unique_date in df_train["datetime"].unique():
    # Filtrer le DataFrame df_historical_weather pour la date unique actuelle
    df_weather_filtered = df_historical_weather[df_historical_weather["datetime"] == unique_date]
    
    # Vérifier s'il y a des données pour cette date
    if len(df_weather_filtered) != 0:
        # Extraire la température pour cette date
        temperature_value = df_weather_filtered.iloc[0]["temperature"]
        
        # Filtrer le DataFrame df_train pour la date unique actuelle
        df_train_filtered = df_train[df_train["datetime"] == unique_date]
        
        # Mettre à jour la colonne "temperature" pour toutes les lignes correspondant à cette date dans df_train
        df_train.loc[df_train_filtered.index, "temperature"] = temperature_value


C:\Users\Jawad\AppData\Local\Temp\ipykernel_14556\4121195767.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-4.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "temperature"] = temperature_value


In [110]:
df_train

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,price_elec,temperature
1414989,11,0,3,790.836,1,2022-11-24 01:00:00,449,1414989,45,199.99,-4.8
953684,9,1,3,4.396,0,2022-07-03 21:00:00,305,953684,37,211.31,19.0
103626,4,0,3,0.000,0,2021-10-06 05:00:00,35,103626,16,98.25,8.7
1745132,15,1,1,0.000,0,2023-03-06 19:00:00,551,1745132,59,129.23,-4.0
1961232,7,1,1,0.000,0,2023-05-14 02:00:00,620,1961232,29,85.59,9.4
...,...,...,...,...,...,...,...,...,...,...,...
786780,0,0,2,0.000,0,2022-05-12 23:00:00,253,786780,1,54.10,10.0
914348,0,0,2,59.062,0,2022-06-21 15:00:00,293,914348,1,199.04,12.5
833853,4,1,0,902.756,1,2022-05-27 14:00:00,268,833853,17,109.55,7.6
188631,14,1,3,1935.450,1,2021-11-03 07:00:00,63,188631,56,119.38,5.3


In [111]:
# Supposons que df_train soit votre DataFrame

# Convertir la colonne "datetime" en type de données datetime si ce n'est pas déjà fait
df_train["datetime"] = pd.to_datetime(df_train["datetime"])

# Créer les nouvelles colonnes "year", "month", "day" et "hour" en tant que nombres
df_train["year"] = df_train["datetime"].dt.year.astype(int)
df_train["month"] = df_train["datetime"].dt.month.astype(int)
df_train["day"] = df_train["datetime"].dt.day.astype(int)
df_train["hour"] = df_train["datetime"].dt.hour.astype(int)

In [109]:

# # Convertir le champ "year" en int
# df_train["year"] = df_train["year"].astype(int)

# # Séparer le champ "date" en "day" et "month" en int
# df_train[['day', 'month', 'year']] = df_train["date"].str.split('-', expand=True).astype(int)


# # Convertir le champ "hour" en int
# df_train["hour"] = df_train["hour"].astype(int)



KeyError: 'year'

In [113]:
# Supprimer la colonne "datetime" si elle n'est plus nécessaire
df_train = df_train.drop('datetime', axis=1)

In [114]:
df_train

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,month,day,hour
1414989,11,0,3,790.836,1,449,1414989,45,199.99,-4.8,2022,11,24,1
953684,9,1,3,4.396,0,305,953684,37,211.31,19.0,2022,7,3,21
103626,4,0,3,0.000,0,35,103626,16,98.25,8.7,2021,10,6,5
1745132,15,1,1,0.000,0,551,1745132,59,129.23,-4.0,2023,3,6,19
1961232,7,1,1,0.000,0,620,1961232,29,85.59,9.4,2023,5,14,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786780,0,0,2,0.000,0,253,786780,1,54.10,10.0,2022,5,12,23
914348,0,0,2,59.062,0,293,914348,1,199.04,12.5,2022,6,21,15
833853,4,1,0,902.756,1,268,833853,17,109.55,7.6,2022,5,27,14
188631,14,1,3,1935.450,1,63,188631,56,119.38,5.3,2021,11,3,7


In [116]:
# # Supposons que df_train et df_gas_prices soient vos DataFrames

# # Calculer la moyenne des prix de df_gas_prices par data_block_id
# average_gas_prices = df_gas_prices.groupby('data_block_id')[['lowest_price_per_mwh', 'highest_price_per_mwh']].mean().reset_index()

# # Fusionner df_train avec la moyenne des prix de df_gas_prices
# df_train = pd.merge(df_train, average_gas_prices, on='data_block_id', how='left')

# # Renommer les colonnes résultantes
# df_train.rename(columns={
#     'lowest_price_per_mwh': 'average_lowest_gas_price',
#     'highest_price_per_mwh': 'average_highest_gas_price'
# }, inplace=True)




In [117]:
df_train

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec,temperature,year,month,day,hour,average_lowest_gas_price,average_highest_gas_price
0,11,0,3,790.836,1,449,1414989,45,199.99,-4.8,2022,11,24,1,121.00,137.99
1,9,1,3,4.396,0,305,953684,37,211.31,19.0,2022,7,3,21,146.50,155.00
2,4,0,3,0.000,0,35,103626,16,98.25,8.7,2021,10,6,5,65.11,66.60
3,15,1,1,0.000,0,551,1745132,59,129.23,-4.0,2023,3,6,19,51.00,54.45
4,7,1,1,0.000,0,620,1961232,29,85.59,9.4,2023,5,14,2,38.60,41.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0,0,2,0.000,0,253,786780,1,54.10,10.0,2022,5,12,23,98.00,104.12
499996,0,0,2,59.062,0,293,914348,1,199.04,12.5,2022,6,21,15,110.05,125.00
499997,4,1,0,902.756,1,268,833853,17,109.55,7.6,2022,5,27,14,102.00,121.95
499998,14,1,3,1935.450,1,63,188631,56,119.38,5.3,2021,11,3,7,59.00,87.90


EXPORT 

In [118]:
df_train.to_csv("./data_transformed/train_500_2.csv", index=False)

In [119]:
df_train.isna().sum()

county                         0
is_business                    0
product_type                   0
target                       129
is_consumption                 0
data_block_id                  0
row_id                         0
prediction_unit_id             0
price_elec                     0
temperature                    0
year                           0
month                          0
day                            0
hour                           0
average_lowest_gas_price     715
average_highest_gas_price    715
dtype: int64